In [ ]:
import zipfile

with zipfile.ZipFile("heavy_makeup_CelebA.zip","r") as zip_file:
  zip_file.extractall("heavy_makeup_CelebA")

In [ ]:
%load_ext tensorboard

#加載 TensorBoard 擴展

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
import tensorboard

In [ ]:
!rm -rf ./logs/
#移除之前產生的任何日誌

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 定義資料路徑
train_data_dir = 'heavy_makeup_CelebA/heavy_makeup_CelebA/train'
validation_data_dir = 'heavy_makeup_CelebA/heavy_makeup_CelebA/val'


# 定義圖片參數
img_width, img_height = 64, 64
batch_size = 32
num_epochs = 3

# 使用ImageDataGenerator來讀取和增強圖片資料
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=(0.8, 1.2),
    channel_shift_range=20,
    vertical_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

# 讀取訓練資料和驗證資料
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [ ]:
# 建立模型
class Model3(tf.keras.Model):
    def __init__(self):
        super(Model3, self).__init__()

        self.conv_block_1 = tf.keras.Sequential([
            tf.keras.layers.Conv2D(filters=16,kernel_size=(3, 3), strides=1, padding='same', activation='relu', input_shape=((64,64,3))),
            tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), strides=1, padding='same', activation='relu'),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)
            #tf.keras.layers.BatchNormalization()，需搭配 training =True, 測試需要關掉(通常在激活函數之前進行)
            #tf.keras.activations.relu()
        ])

        self.conv_block_2 = tf.keras.Sequential([
            tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=1, padding='same', activation='relu'),
            tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=1, padding='same', activation='relu'),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)])

        self.classifier = tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=1,activation='sigmoid')
        ])

    def call(self, inputs):
        inputs = tf.cast(inputs, dtype=tf.float32)  # 將輸入轉換為 float32
        x = self.conv_block_1(inputs)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x


# 編譯模型

model =Model3()

model.compile(optimizer='Adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])




Found 2000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [ ]:
logdir ="./logs"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    callbacks=[tensorboard_callback]
)

In [ ]:
%tensorboard --logdir logs --port 6008

In [ ]:
u = validation_generator[0][0][:1]
u.shape

(1, 64, 64, 3)

In [ ]:
import numpy as np

c = model.predict(u)
c

1/1 [==============================] - 0s 130ms/step


array([[0.5882347]], dtype=float32)

In [ ]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [ ]:
model.save('./model')

In [ ]:
loaded_model = tf.keras.models.load_model('./model')

In [ ]:
loaded_model.summary()

In [ ]:
base = tf.keras.applications.MobileNetV2(input_shape=(64, 64, 3), include_top=False, weights='imagenet')
base.trainable = False
net = tf.keras.layers.GlobalAveragePooling2D()(base.output)
net = tf.keras.layers.Dense(1,activation='sigmoid')(net)

model1 = tf.keras.Model(inputs=[base.input], outputs=[net])

In [ ]:
model1.compile(optimizer='Adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])


In [ ]:
history = model1.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator)

In [ ]:
plt.plot(range(0,len(history.history['loss'])),history.history['loss'],label='train_cost')
plt.plot(range(0,len(history.history['val_loss'])),history.history['val_loss'],label='test_cost')
plt.legend()